In [1]:
!pip install featuretools
import pandas as pd
import featuretools as ft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 23.7 MB/s eta 0:00:00


In [3]:
!pip install google-cloud-bigquery

from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()


In [4]:
# Project and dataset details
project_id = 'dmw-individual-1'
dataset_id = 'e_commerce'

# Table references
table_customers = f'{project_id}.{dataset_id}.Customers'
table_products = f'{project_id}.{dataset_id}.Products'
table_orders = f'{project_id}.{dataset_id}.Orders'
table_order_details = f'{project_id}.{dataset_id}.OrderDetails'

# Load each table into a pandas DataFrame
query_customers = f'SELECT * FROM `{table_customers}`'
df_customers = client.query(query_customers).to_dataframe()

query_products = f'SELECT * FROM `{table_products}`'
df_products = client.query(query_products).to_dataframe()

query_orders = f'SELECT * FROM `{table_orders}`'
df_orders = client.query(query_orders).to_dataframe()

query_order_details = f'SELECT * FROM `{table_order_details}`'
df_order_details = client.query(query_order_details).to_dataframe()


In [5]:
# Check the data
print(df_customers.head())
print(df_products.head())
print(df_orders.head())
print(df_order_details.head())


   CustomerID           Name                     Email  SignupDate  \
0         111     Kelen Khor     kelenkhor@example.com  2023-12-31   
1         102    Badrul Baba    badrulbaba@example.com  2023-11-12   
2         108  Hairul Azreen  hairulazreen@example.com  2023-11-20   
3         110      Jason Ong      jasonong@example.com  2023-01-04   
4         120  Turu Arumugan  turuarumugan@example.com  2023-08-21   

   TotalOrders  TotalItemsPurchased  TotalDiscountReceived MostCommonOrderDay  \
0            5                 16.0                   0.40            Tuesday   
1            3                 11.0                   0.10             Monday   
2            1                  2.0                   0.15             Friday   
3            4                  7.0                   0.20            Tuesday   
4            4                 11.0                   0.15          Wednesday   

  MostCommonOrderMonth  row_number  
0                March          12  
1                 

In [6]:
df_customers['SignupDate'] = pd.to_datetime(df_customers['SignupDate'], format='%Y-%m-%d')
df_orders['OrderDate'] = pd.to_datetime(df_orders['OrderDate'], format='%Y-%m-%d')
df_orders['ShipDate'] = pd.to_datetime(df_orders['ShipDate'], format='%Y-%m-%d')


In [7]:
print(df_customers['CustomerID'].isnull().sum())
df_customers = df_customers.dropna(subset=['CustomerID'])

0


In [8]:
es = ft.EntitySet('ecommerce')

es.add_dataframe(dataframe_name='customers', dataframe=df_customers, time_index='SignupDate')
es.add_dataframe(dataframe_name='products', dataframe=df_products, index='ProductID')
es.add_dataframe(dataframe_name='orders', dataframe=df_orders, time_index='OrderDate')
es.add_dataframe(dataframe_name='order_details', dataframe=df_order_details, make_index=True, index='OrderDetailID')

# Define Relationships
es.add_relationship('customers', 'CustomerID', 'orders', 'CustomerID')
es.add_relationship('orders', 'OrderID', 'order_details', 'OrderID')
es.add_relationship('products', 'ProductID', 'order_details', 'ProductID')

/usr/local/lib/python3.10/dist-packages/featuretools/entityset/entityset.py:1717: UserWarning: Using first column as index. To change this, specify the index parameter
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_dateti

Entityset: ecommerce
  DataFrames:
    customers [Rows: 20, Columns: 10]
    products [Rows: 10, Columns: 4]
    orders [Rows: 50, Columns: 4]
    order_details [Rows: 50, Columns: 5]
  Relationships:
    orders.CustomerID -> customers.CustomerID
    order_details.OrderID -> orders.OrderID
    order_details.ProductID -> products.ProductID

In [9]:
#Display all columns and rows for the generated features
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Run Deep Feature Synthesis
features, feature_defs = ft.dfs(entityset=es, target_dataframe_name="customers", agg_primitives=["sum", "mean","mode", "count"])
# Add CustomerID back as a column
features.reset_index(inplace=True)
features

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x7f0df96f1bd0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7f0df96f0ca0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x7f0df96f1bd0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable w

,CustomerID,TotalOrders,TotalItemsPurchased,TotalDiscountReceived,row_number,COUNT(orders),COUNT(order_details),MEAN(order_details.Discount),MEAN(order_details.Quantity),SUM(order_details.Discount),SUM(order_details.Quantity),DAY(SignupDate),MONTH(SignupDate),WEEKDAY(SignupDate),YEAR(SignupDate),MEAN(orders.COUNT(order_details)),MEAN(orders.MEAN(order_details.Discount)),MEAN(orders.MEAN(order_details.Quantity)),MEAN(orders.SUM(order_details.Discount)),MEAN(orders.SUM(order_details.Quantity)),MODE(orders.DAY(OrderDate)),MODE(orders.DAY(ShipDate)),MODE(orders.MONTH(OrderDate)),MODE(orders.MONTH(ShipDate)),MODE(orders.WEEKDAY(OrderDate)),MODE(orders.WEEKDAY(ShipDate)),MODE(orders.YEAR(OrderDate)),MODE(orders.YEAR(ShipDate)),SUM(orders.MEAN(order_details.Discount)),SUM(orders.MEAN(order_details.Quantity))
0,110,4,7.0,0.20,17,4,4,0.050000,1.750000,0.20,7.0,4,1,2,2023,1.0,0.050000,1.750000,0.050000,1.750000,3,13,1,1,1,1,2023,2023,0.20,7.0
1,115,4,15.0,0.25,11,4,4,0.062500,3.750000,0.25,15.0,5,1,3,2023,1.0,0.062500,3.750000,0.062500,3.750000,31,2,3,1,1,1,2023,2023,0.25,15.0
2,109,4,18.0,0.45,8,4,4,0.112500,4.500000,0.45,18.0,19,2,6,2023,1.0,0.112500,4.500000,0.112500,4.500000,31,1,6,1,0,2,2023,2023,0.45,18.0
3,105,1,2.0,0.05,13,1,1,0.050000,2.000000,0.05,2.0,28,3,1,2023,1.0,0.050000,2.000000,0.050000,2.000000,19,20,8,8,5,6,2023,2023,0.05,2.0
4,104,1,3.0,0.00,15,1,1,0.000000,3.000000,0.00,3.0,27,4,3,2023,1.0,0.000000,3.000000,0.000000,3.000000,8,13,1,1,6,4,2023,2023,0.00,3.0
5,103,0,0.0,0.00,3,0,0,NaN,NaN,0.00,0.0,30,4,6,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
6,118,1,4.0,0.00,21,1,1,0.000000,4.000000,0.00,4.0,25,6,6,2023,1.0,0.000000,4.000000,0.000000,4.000000,19,25,11,11,6,5,2023,2023,0.00,4.0
7,117,4,9.0,0.35,19,4,4,0.087500,2.250000,0.35,9.0,8,8,1,2023,1.0,0.087500,2.250000,0.087500,2.250000,2,6,1,1,0,0,2023,2023,0.35,9.0
8,107,1,1.0,0.15,4,1,1,0.150000,1.000000,0.15,1.0,11,8,4,2023,1.0,0.150000,1.000000,0.150000,1.000000,25,30,5,5,3,1,2023,2023,0.15,1.0
9,116,1,1.0,0.00,9,1,1,0.000000,1.000000,0.00,1.0,14,8,0,2023,1.0,0.000000,1.000000,0.000000,1.000000,27,3,3,4,0,0,2023,2023,0.00,1.0


In [10]:
# View the Feature Matrix
print("Total Features: %d" % len(feature_defs))
for feature in features.columns:
  print(feature)

Total Features: 29
CustomerID
TotalOrders
TotalItemsPurchased
TotalDiscountReceived
row_number
COUNT(orders)
COUNT(order_details)
MEAN(order_details.Discount)
MEAN(order_details.Quantity)
SUM(order_details.Discount)
SUM(order_details.Quantity)
DAY(SignupDate)
MONTH(SignupDate)
WEEKDAY(SignupDate)
YEAR(SignupDate)
MEAN(orders.COUNT(order_details))
MEAN(orders.MEAN(order_details.Discount))
MEAN(orders.MEAN(order_details.Quantity))
MEAN(orders.SUM(order_details.Discount))
MEAN(orders.SUM(order_details.Quantity))
MODE(orders.DAY(OrderDate))
MODE(orders.DAY(ShipDate))
MODE(orders.MONTH(OrderDate))
MODE(orders.MONTH(ShipDate))
MODE(orders.WEEKDAY(OrderDate))
MODE(orders.WEEKDAY(ShipDate))
MODE(orders.YEAR(OrderDate))
MODE(orders.YEAR(ShipDate))
SUM(orders.MEAN(order_details.Discount))
SUM(orders.MEAN(order_details.Quantity))


In [11]:
# Define the relevant features to include
selected_features = [
    'CustomerID',               # Primary key for alignment
    'COUNT(orders)',            # Total orders
    'SUM(order_details.Quantity)',  # Total items purchased
    'SUM(order_details.Discount)',  # Total discount received
    'MODE(orders.MONTH(OrderDate))', # Most common order month
    'MODE(orders.WEEKDAY(OrderDate))'  # Most common order day
]

# Filter the feature matrix
filtered_feature_matrix = features[selected_features]


In [12]:
# Rename columns for clarity and compatibility
filtered_feature_matrix.rename(columns={
    'COUNT(orders)': 'TotalOrders',
    'SUM(order_details.Quantity)': 'TotalItemsPurchased',
    'SUM(order_details.Discount)': 'TotalDiscountReceived',
    'MODE(orders.MONTH(OrderDate))': 'MostCommonOrderMonth',
    'MODE(orders.WEEKDAY(OrderDate))': 'MostCommonOrderDay'
}, inplace=True)


In [13]:
# Rename one column
filtered_feature_matrix.rename(columns={'Most common order month': 'MostCommonOrderMonth'}, inplace=True)

In [14]:
# Fill missing values
filtered_feature_matrix.fillna({
    'TotalOrders': 0,
    'TotalItemsPurchased': 0,
    'TotalDiscountReceived': 0.0,
}, inplace=True)


In [15]:
import calendar

# Map weekday numbers to names
filtered_feature_matrix['MostCommonOrderDay'] = filtered_feature_matrix['MostCommonOrderDay'].map(lambda x: calendar.day_name[int(x)] if pd.notnull(x) else None)


In [16]:
# Map month numbers to names
filtered_feature_matrix['MostCommonOrderMonth'] = filtered_feature_matrix['MostCommonOrderMonth'].map(lambda x: calendar.month_name[int(x)] if pd.notnull(x) else None)

print(filtered_feature_matrix[['MostCommonOrderDay', 'MostCommonOrderMonth']].head())


  MostCommonOrderDay MostCommonOrderMonth
0            Tuesday              January
1            Tuesday                March
2             Monday                 June
3           Saturday               August
4             Sunday              January


In [17]:
print(filtered_feature_matrix.head())
filtered_feature_matrix.to_csv('filtered_feature_matrix.csv', index=False)

   CustomerID  TotalOrders  TotalItemsPurchased  TotalDiscountReceived  \
0         110            4                  7.0                   0.20   
1         115            4                 15.0                   0.25   
2         109            4                 18.0                   0.45   
3         105            1                  2.0                   0.05   
4         104            1                  3.0                   0.00   

  MostCommonOrderMonth MostCommonOrderDay  
0              January            Tuesday  
1                March            Tuesday  
2                 June             Monday  
3               August           Saturday  
4              January             Sunday  


In [18]:
from google.cloud import bigquery

client = bigquery.Client()

# Define the temporary table
temp_table_id = 'dmw-individual-1.e_commerce.filtered_feature_matrix'

# Upload the selected features
job = client.load_table_from_dataframe(filtered_feature_matrix, temp_table_id)
job.result()  # Wait for the upload to complete

print(f"Uploaded selected features to temporary table {temp_table_id}")


Uploaded selected features to temporary table dmw-individual-1.e_commerce.filtered_feature_matrix
